# The ABC of ATB (Automated Trading Backtest)

## A. Obtaining Data

In this part, we first retrieve data and save them as CSV files.We will first start with the easiest way to obtain data from Yahoo finance, and then look at other data sources in the future.

### A1. Getting Data from Yahoo Finance

In [5]:
"""
This code simply gets data from Yahoo Finance
Input: symbols (list), start and end time
Output: CVS forms contains the time series of a stock
        for each symbol in the list
"""

# For compatibility with Py2 and Py3
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

# Import necessary functions
import datetime
import pandas as pd
import pandas_datareader.data as web
import sys

# Check Python and Pandas version
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

# Inputs
symbols = ['SPY', 'MAR', 'AMZN']
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2017, 8, 16)

if __name__ == "__main__":
    for s in symbols:
        # Getting the data
        data = web.DataReader(s, 'yahoo', start, end)
        # Set the file name
        filename = s + '.csv'
        # Write to CSV file
        pd.DataFrame.to_csv(data, filename, index=False)
    print('Done.')

Python version 3.5.3 |Anaconda 4.0.0 (64-bit)| (default, Feb 22 2017, 21:28:42) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.18.0


ConnectionError: HTTPConnectionPool(host='ichart.finance.yahoo.com', port=80): Max retries exceeded with url: /table.csv?c=2010&e=16&g=d&a=0&f=2017&d=7&s=SPY&ignore=.csv&b=1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x00000236FDD5FF28>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))